In [18]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from resnets_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

### Load the data set

In [19]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [20]:
img_height,img_width = 64,64 
num_classes = 6
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

/home/priya/miniconda3/envs/exptt/lib/python3.6/site-packages/Keras_Applications-1.0.6-py3.6.egg/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [21]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation= 'relu')(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

# freeze all layers of base model
for layer in base_model.layers[:6]:
    layer.trainable = False
for layer in base_model.layers[6:]: #let's start training some last layers
    layer.trainable = True
    
for i,layer in enumerate(base_model.layers):
    print(i, layer, layer.trainable)

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [23]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 64)

Epoch 1/100
1080/1080 [==============================] - 8s 7ms/step - loss: 2.9151 - acc: 0.1648
Epoch 2/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.7965 - acc: 0.2500
Epoch 3/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.7302 - acc: 0.2750
Epoch 4/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.6956 - acc: 0.2602
Epoch 5/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.6415 - acc: 0.3037
Epoch 6/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.5727 - acc: 0.3463
Epoch 7/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.4862 - acc: 0.3704
Epoch 8/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.4121 - acc: 0.4259
Epoch 9/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.3466 - acc: 0.4417
Epoch 10/100
1080/1080 [==============================] - 1s 1ms/step - loss: 1.2809 - acc: 0.5167
Epoch 11/100
1080/1

1080/1080 [==============================] - 1s 1ms/step - loss: 0.0321 - acc: 0.9852
Epoch 84/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0384 - acc: 0.9935
Epoch 85/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0242 - acc: 0.9954
Epoch 86/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0367 - acc: 0.9880
Epoch 87/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0612 - acc: 0.9833
Epoch 88/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0794 - acc: 0.9759
Epoch 89/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0410 - acc: 0.9917
Epoch 90/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0423 - acc: 0.9889
Epoch 91/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0607 - acc: 0.9815
Epoch 92/100
1080/1080 [==============================] - 1s 1ms/step - loss: 0.0507 - acc: 0.9880
Epoch 93/100
1080/1080 

In [25]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

120/120 [==============================] - 1s 10ms/step
Loss = 1.1325975100199381
Test Accuracy = 0.7833333333333333
